In [ ]:
This notebook is to explore various prediction models from the Used Car Price data 
at : https://www.kaggle.com/CooperUnion/cardataset
Various Regression techniques are explored, with K-fold cross validation, grid search of parameters, including 
Deep Learning techniques and the best approach is selected based on Accuracy

In [1]:
#Import all necessary libraries
import pickle
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from keras.models import Sequential
from keras.layers import Dense   
from keras import optimizers

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [ ]:
Read the pre-processed pkl files

In [6]:
train_X_ordinary=pd.read_pickle('C:/users/hackuser1/Hackathon18/train_X_ord.pkl')
test_X_ordinary=pd.read_pickle('C:/users/hackuser1/Hackathon18/test_X_ord.pkl')
train_Y_ordinary=pd.read_pickle('C:/users/hackuser1/Hackathon18/train_Y_ord.pkl')
test_Y_ordinary=pd.read_pickle('C:/users/hackuser1/Hackathon18/test_Y_ord.pkl')

train_X_ord_make=pd.read_pickle('C:/users/hackuser1/Hackathon18/train_X_ord.pkl')
test_X_ord_make=pd.read_pickle('C:/users/hackuser1/Hackathon18/test_X_ord.pkl')
train_Y_ord_make=pd.read_pickle('C:/users/hackuser1/Hackathon18/train_Y_ord.pkl')
test_Y_ord_make=pd.read_pickle('C:/users/hackuser1/Hackathon18/test_Y_ord.pkl')


In [42]:
#fit XGBRegressor and use this to make predictions on the test data and check the RMSE

xgboost_reg = XGBRegressor()
xgboost_reg.fit(train_X,train_Y)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [43]:
carSales_predictions = xgboost_reg.predict(test_X)
xgboost_mse = mean_squared_error(test_Y, carSales_predictions)
xgboost_rmse = np.sqrt(xgboost_mse)
print(xgboost_rmse)

4111.19
